# Automated License Plate Detection using YOLOv8 and EasyOCR

This notebook implements an automated license plate detection pipeline
that combines a YOLOv8-based object detection model with EasyOCR for
optical character recognition.

The system processes uploaded video inputs, detects vehicles and their
corresponding license plates, overlays recognized text on the output
video, and exports the annotated results for further analysis.

**Execution Environment:** Google Colab (GPU-enabled)


In [ ]:
class LicensePlateDetector:
    """
    License plate detection and database management system
    Detects license plates from images and manages a clean database
    """

    def __init__(self):
        self.setup_dependencies()
        self.database = []  # List of detected license plates
        self.duplicate_entries = []
        self.invalid_entries = []

    def setup_dependencies(self):
        """Setup and install required dependencies"""
        print("Setting up dependencies...")
    def preprocess_image(self, image):
        """Preprocess image for better OCR results"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply bilateral filter to reduce noise while preserving edges
        denoised = cv2.bilateralFilter(gray, 11, 17, 17)

        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(
            denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2
        )

        # Apply morphological operations to clean up
        kernel = np.ones((3, 3), np.uint8)
        morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        return gray, thresh, morph

    def detect_license_plate(self, image, image_name=""):
        """Detect license plate text using OCR"""
        if self.ocr_reader is None:
            return None

        try:
            # Preprocess image
            gray, thresh, morph = self.preprocess_image(image)

            # Try OCR on original image
            results_original = self.ocr_reader.readtext(image)

            # Try OCR on grayscale
            results_gray = self.ocr_reader.readtext(gray)

            # Try OCR on thresholded image
            results_thresh = self.ocr_reader.readtext(thresh)

            # Combine all results
            all_results = results_original + results_gray + results_thresh

            # Extract text with confidence > 0.2
            detected_texts = []
            for (bbox, text, confidence) in all_results:
                if confidence > 0.2:
                    cleaned_text = self.clean_license_plate_text(text)
                    if cleaned_text:
                        detected_texts.append({
                            'text': cleaned_text,
                            'confidence': confidence,
                            'bbox': bbox
                        })

            # Sort by confidence and return best result
            if detected_texts:
                detected_texts.sort(key=lambda x: x['confidence'], reverse=True)
                return detected_texts[0]['text'], detected_texts[0]['confidence']

        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")

        return None, 0

    def clean_license_plate_text(self, text):
        """Clean and validate license plate text"""
        # Remove spaces and special characters, keep only alphanumeric
        cleaned = ''.join(c for c in text if c.isalnum()).upper()

        # License plates typically have 3-15 characters
        if 3 <= len(cleaned) <= 15:
            return cleaned

        return None

    def is_valid_license_plate(self, text):
        """Validate if text looks like a license plate"""
        if not text or len(text) < 3:
            return False

        # Should contain at least one letter and one number
        has_letter = any(c.isalpha() for c in text)
        has_number = any(c.isdigit() for c in text)

        # Should not be too long
        if len(text) > 15:
            return False

        # Should not be all numbers or all letters (most plates have both)
        if text.isalpha() or text.isdigit():
            return False

        return has_letter and has_number
    def process_images(self, image_paths):
        """Process multiple images and build database"""
        print(f"\n{'='*60}")
        print("PROCESSING LICENSE PLATE IMAGES")
        print(f"{'='*60}\n")

        results = []

        for i, img_path in enumerate(image_paths):
            print(f"[{i+1}/{len(image_paths)}] Processing: {os.path.basename(img_path)}")

            # Read image
            image = cv2.imread(img_path)
            if image is None:
                print(f"  ✗ Could not read image")
                self.invalid_entries.append({
                    'image': os.path.basename(img_path),
                    'reason': 'Could not read image file'
                })
                continue

            # Detect license plate
            plate_text, confidence = self.detect_license_plate(image, os.path.basename(img_path))

            if plate_text:
                print(f"  ✓ Detected: {plate_text} (confidence: {confidence:.2f})")

                # Check validity
                if not self.is_valid_license_plate(plate_text):
                    print(f"  ⚠ Invalid format - skipping")
                    self.invalid_entries.append({
                        'image': os.path.basename(img_path),
                        'text': plate_text,
                        'reason': 'Invalid license plate format'
                    })
                    continue

                # Check for duplicates
                existing = [entry for entry in self.database if entry['plate_number'] == plate_text]
                if existing:
                    print(f"  ⚠ Duplicate detected!")
                    self.duplicate_entries.append({
                        'plate_number': plate_text,
                        'images': [existing[0]['image'], os.path.basename(img_path)],
                        'confidence': [existing[0]['confidence'], confidence]
                    })
                else:
                    # Add to database
                    self.database.append({
                        'plate_number': plate_text,
                        'image': os.path.basename(img_path),
                        'confidence': confidence,
                        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                    })
            else:
                print(f"  ✗ No license plate detected")
                self.invalid_entries.append({
                    'image': os.path.basename(img_path),
                    'reason': 'No text detected'
                })

        print(f"\n{'='*60}")
        print("PROCESSING COMPLETE")
        print(f"{'='*60}\n")

        return results